In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import re
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()


/home/youwei2/miniconda3/envs/ds/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [3]:
df = pd.read_csv('data_preprocessing.csv')
df

,startphrase,ending0,ending1,ending2,ending3,label
0,The man plays continuously around on the drum ...,continues to talk as he begins pressing the gu...,continues playing the piano while the boy cont...,finishes playing the drum and pauses before th...,parts the flip and covers the equipment he made.,2
1,Another man on the field catches the ball. The...,swimming in him and cheers him on.,makes several pictures and throws a ball.,grabs a mallet and throws the ball.,is falling near the pool in attempt.,2
2,"Someone sits across from him, looking small an...",reaches out and takes someone's hand.,is soaked through very bare curtains.,puts his leg around someone's neck.,is quiet - eyed with woman.,0
3,"Someone sits across from him, looking small an...",is soaked through very bare curtains.,puts his leg around someone's neck.,"stands, ready to usher someone out.",is quiet - eyed with woman.,2
4,Someone's gaze is steady. His high white shirt...,are high and warm.,hangs on his wrists.,drops out of view.,is shaven and bald.,0
...,...,...,...,...,...,...
294,A woman is standing inside her kitchen. She,applies soap and shaving cream to her face.,is talking to the camera in the kitchen.,shows how to wrap a gift around present.,starts talking to the camera in the bathroom.,1
295,An arrow goes through the hole of a donut. An ...,pulls a hole on the ground.,is pulled out with a knife.,cuts the string holding an apple.,is attached to a released point.,2
296,Another child moves around behind her. More ch...,moves around and fourth.,hops to the side.,laying down looking down.,proceeds to smile slightly.,3
297,The woman exercise on front the ocean spinning...,finish watching the other woman perform.,spins the hoop with her hand.,lifts the weight and spins around.,"begins riding a heavy, elliptical bike.",1


In [4]:
def score_ending(startphrase, ending):
    input_text = startphrase + " " + ending
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(inputs, labels=inputs)
    loss = outputs.loss.item()
    return loss

def pick_best_ending_for_df(df):
    best_endings = []
    for index, row in df.iterrows():
        startphrase = row['startphrase']
        endings = [row['ending0'], row['ending1'], row['ending2'], row['ending3']]
        scores = [score_ending(startphrase, ending) for ending in endings]
        best_ending_idx = scores.index(min(scores))
        best_ending = endings[best_ending_idx]
        best_endings.append(best_ending)
    return best_endings

df['best_ending'] = pick_best_ending_for_df(df)

In [5]:
df

,startphrase,ending0,ending1,ending2,ending3,label,best_ending
0,The man plays continuously around on the drum ...,continues to talk as he begins pressing the gu...,continues playing the piano while the boy cont...,finishes playing the drum and pauses before th...,parts the flip and covers the equipment he made.,2,continues playing the piano while the boy cont...
1,Another man on the field catches the ball. The...,swimming in him and cheers him on.,makes several pictures and throws a ball.,grabs a mallet and throws the ball.,is falling near the pool in attempt.,2,grabs a mallet and throws the ball.
2,"Someone sits across from him, looking small an...",reaches out and takes someone's hand.,is soaked through very bare curtains.,puts his leg around someone's neck.,is quiet - eyed with woman.,0,reaches out and takes someone's hand.
3,"Someone sits across from him, looking small an...",is soaked through very bare curtains.,puts his leg around someone's neck.,"stands, ready to usher someone out.",is quiet - eyed with woman.,2,puts his leg around someone's neck.
4,Someone's gaze is steady. His high white shirt...,are high and warm.,hangs on his wrists.,drops out of view.,is shaven and bald.,0,hangs on his wrists.
...,...,...,...,...,...,...,...
294,A woman is standing inside her kitchen. She,applies soap and shaving cream to her face.,is talking to the camera in the kitchen.,shows how to wrap a gift around present.,starts talking to the camera in the bathroom.,1,is talking to the camera in the kitchen.
295,An arrow goes through the hole of a donut. An ...,pulls a hole on the ground.,is pulled out with a knife.,cuts the string holding an apple.,is attached to a released point.,2,is pulled out with a knife.
296,Another child moves around behind her. More ch...,moves around and fourth.,hops to the side.,laying down looking down.,proceeds to smile slightly.,3,hops to the side.
297,The woman exercise on front the ocean spinning...,finish watching the other woman perform.,spins the hoop with her hand.,lifts the weight and spins around.,"begins riding a heavy, elliptical bike.",1,spins the hoop with her hand.


In [6]:
#calculate suprisal:
def calculate_surprisal(sentence):
    inputs = tokenizer(sentence, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        logits = outputs.logits

    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
    surprisal = -torch.sum(log_probs * torch.nn.functional.one_hot(input_ids, num_classes=logits.size(-1)).float())
    
    return surprisal.item()

In [7]:
columns=['ending0', 'ending1', 'ending2', 'ending3','best_ending']

for index,row in df.iterrows():
    for col in columns:
        sentence = row[col]
        surprisal = calculate_surprisal(sentence)
        df.at[index,col] = surprisal
df

,startphrase,ending0,ending1,ending2,ending3,label,best_ending
0,The man plays continuously around on the drum ...,106.062126,105.51252,96.884735,90.222755,2,105.51252
1,Another man on the field catches the ball. The...,86.585938,79.253586,99.089836,70.123375,2,99.089836
2,"Someone sits across from him, looking small an...",85.449265,55.319748,85.888588,61.42556,0,85.449265
3,"Someone sits across from him, looking small an...",55.319748,85.888588,87.065315,61.42556,2,85.888588
4,Someone's gaze is steady. His high white shirt...,42.692604,53.533142,48.238132,56.046124,0,53.533142
...,...,...,...,...,...,...,...
294,A woman is standing inside her kitchen. She,86.396469,75.682159,77.956757,84.997215,1,75.682159
295,An arrow goes through the hole of a donut. An ...,74.971596,63.508228,62.587616,57.772198,2,63.508228
296,Another child moves around behind her. More ch...,50.764687,44.882004,57.364052,64.445938,3,44.882004
297,The woman exercise on front the ocean spinning...,70.271095,72.533035,68.238945,108.430801,1,72.533035


In [8]:
#compare the accuracy: surprisal one with the model's selection one

columns=['ending0', 'ending1', 'ending2', 'ending3']

def find_label(row):
    smallest_col = min(columns, key=lambda col: row[col])
    return smallest_col

In [9]:
df['surprisal_smallest'] = df.apply(find_label, axis=1)
df

,startphrase,ending0,ending1,ending2,ending3,label,best_ending,surprisal_smallest
0,The man plays continuously around on the drum ...,106.062126,105.51252,96.884735,90.222755,2,105.51252,ending3
1,Another man on the field catches the ball. The...,86.585938,79.253586,99.089836,70.123375,2,99.089836,ending3
2,"Someone sits across from him, looking small an...",85.449265,55.319748,85.888588,61.42556,0,85.449265,ending1
3,"Someone sits across from him, looking small an...",55.319748,85.888588,87.065315,61.42556,2,85.888588,ending0
4,Someone's gaze is steady. His high white shirt...,42.692604,53.533142,48.238132,56.046124,0,53.533142,ending0
...,...,...,...,...,...,...,...,...
294,A woman is standing inside her kitchen. She,86.396469,75.682159,77.956757,84.997215,1,75.682159,ending1
295,An arrow goes through the hole of a donut. An ...,74.971596,63.508228,62.587616,57.772198,2,63.508228,ending3
296,Another child moves around behind her. More ch...,50.764687,44.882004,57.364052,64.445938,3,44.882004,ending1
297,The woman exercise on front the ocean spinning...,70.271095,72.533035,68.238945,108.430801,1,72.533035,ending2


In [10]:
columns=['ending0', 'ending1', 'ending2', 'ending3']

def find_label(row):
    largest_col = max(columns, key=lambda col: row[col])
    return largest_col

In [11]:
df['surprisal_largest'] = df.apply(find_label, axis=1)
df

,startphrase,ending0,ending1,ending2,ending3,label,best_ending,surprisal_smallest,surprisal_largest
0,The man plays continuously around on the drum ...,106.062126,105.51252,96.884735,90.222755,2,105.51252,ending3,ending0
1,Another man on the field catches the ball. The...,86.585938,79.253586,99.089836,70.123375,2,99.089836,ending3,ending2
2,"Someone sits across from him, looking small an...",85.449265,55.319748,85.888588,61.42556,0,85.449265,ending1,ending2
3,"Someone sits across from him, looking small an...",55.319748,85.888588,87.065315,61.42556,2,85.888588,ending0,ending2
4,Someone's gaze is steady. His high white shirt...,42.692604,53.533142,48.238132,56.046124,0,53.533142,ending0,ending3
...,...,...,...,...,...,...,...,...,...
294,A woman is standing inside her kitchen. She,86.396469,75.682159,77.956757,84.997215,1,75.682159,ending1,ending0
295,An arrow goes through the hole of a donut. An ...,74.971596,63.508228,62.587616,57.772198,2,63.508228,ending3,ending0
296,Another child moves around behind her. More ch...,50.764687,44.882004,57.364052,64.445938,3,44.882004,ending1,ending3
297,The woman exercise on front the ocean spinning...,70.271095,72.533035,68.238945,108.430801,1,72.533035,ending2,ending3


In [12]:
def find_matching_column(row):
    if row['best_ending'] == row['ending0']:
        return 'ending0'
    elif row['best_ending'] == row['ending1']:
        return 'ending1'
    elif row['best_ending'] == row['ending2']:
        return 'ending2'
    elif row['best_ending'] == row['ending3']:
        return 'ending3'
    return None

df['model_select'] = df.apply(find_matching_column, axis=1)

In [13]:
df

,startphrase,ending0,ending1,ending2,ending3,label,best_ending,surprisal_smallest,surprisal_largest,model_select
0,The man plays continuously around on the drum ...,106.062126,105.51252,96.884735,90.222755,2,105.51252,ending3,ending0,ending1
1,Another man on the field catches the ball. The...,86.585938,79.253586,99.089836,70.123375,2,99.089836,ending3,ending2,ending2
2,"Someone sits across from him, looking small an...",85.449265,55.319748,85.888588,61.42556,0,85.449265,ending1,ending2,ending0
3,"Someone sits across from him, looking small an...",55.319748,85.888588,87.065315,61.42556,2,85.888588,ending0,ending2,ending1
4,Someone's gaze is steady. His high white shirt...,42.692604,53.533142,48.238132,56.046124,0,53.533142,ending0,ending3,ending1
...,...,...,...,...,...,...,...,...,...,...
294,A woman is standing inside her kitchen. She,86.396469,75.682159,77.956757,84.997215,1,75.682159,ending1,ending0,ending1
295,An arrow goes through the hole of a donut. An ...,74.971596,63.508228,62.587616,57.772198,2,63.508228,ending3,ending0,ending1
296,Another child moves around behind her. More ch...,50.764687,44.882004,57.364052,64.445938,3,44.882004,ending1,ending3,ending1
297,The woman exercise on front the ocean spinning...,70.271095,72.533035,68.238945,108.430801,1,72.533035,ending2,ending3,ending1


In [14]:
#calculate for accuracy-smallest
comparison = df['surprisal_smallest'] == df['model_select']
correct_predictions = comparison.sum()
accuracy = correct_predictions / len(df)

In [15]:
accuracy

0.27424749163879597

In [16]:
# want to calculate how often label match surprisal_smallest or label with model_select
# try large surprisal 

In [17]:
#calculate for accuracy-largest
comparison = df['surprisal_largest'] == df['model_select']
correct_predictions = comparison.sum()
accuracy = correct_predictions / len(df)

In [18]:
accuracy

0.2608695652173913

In [19]:
df

,startphrase,ending0,ending1,ending2,ending3,label,best_ending,surprisal_smallest,surprisal_largest,model_select
0,The man plays continuously around on the drum ...,106.062126,105.51252,96.884735,90.222755,2,105.51252,ending3,ending0,ending1
1,Another man on the field catches the ball. The...,86.585938,79.253586,99.089836,70.123375,2,99.089836,ending3,ending2,ending2
2,"Someone sits across from him, looking small an...",85.449265,55.319748,85.888588,61.42556,0,85.449265,ending1,ending2,ending0
3,"Someone sits across from him, looking small an...",55.319748,85.888588,87.065315,61.42556,2,85.888588,ending0,ending2,ending1
4,Someone's gaze is steady. His high white shirt...,42.692604,53.533142,48.238132,56.046124,0,53.533142,ending0,ending3,ending1
...,...,...,...,...,...,...,...,...,...,...
294,A woman is standing inside her kitchen. She,86.396469,75.682159,77.956757,84.997215,1,75.682159,ending1,ending0,ending1
295,An arrow goes through the hole of a donut. An ...,74.971596,63.508228,62.587616,57.772198,2,63.508228,ending3,ending0,ending1
296,Another child moves around behind her. More ch...,50.764687,44.882004,57.364052,64.445938,3,44.882004,ending1,ending3,ending1
297,The woman exercise on front the ocean spinning...,70.271095,72.533035,68.238945,108.430801,1,72.533035,ending2,ending3,ending1


In [23]:
# want to calculate how often label match surprisal_smallest or label with model_select
def find_matching(row):
    if row['surprisal_smallest'] == 'ending0':
        return 0
    elif row['surprisal_smallest'] == 'ending1':
        return 1
    elif row['surprisal_smallest'] == 'ending2':
        return 2
    elif row['surprisal_smallest'] == 'ending3':
        return 3
    return None

df['surprisal_smallest_number'] = df.apply(find_matching,axis=1)

In [24]:
df

,startphrase,ending0,ending1,ending2,ending3,label,best_ending,surprisal_smallest,surprisal_largest,model_select,model_vs_human,surprisal_smallest_number
0,The man plays continuously around on the drum ...,106.062126,105.51252,96.884735,90.222755,2,105.51252,ending3,ending0,ending1,3,3
1,Another man on the field catches the ball. The...,86.585938,79.253586,99.089836,70.123375,2,99.089836,ending3,ending2,ending2,3,3
2,"Someone sits across from him, looking small an...",85.449265,55.319748,85.888588,61.42556,0,85.449265,ending1,ending2,ending0,1,1
3,"Someone sits across from him, looking small an...",55.319748,85.888588,87.065315,61.42556,2,85.888588,ending0,ending2,ending1,0,0
4,Someone's gaze is steady. His high white shirt...,42.692604,53.533142,48.238132,56.046124,0,53.533142,ending0,ending3,ending1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
294,A woman is standing inside her kitchen. She,86.396469,75.682159,77.956757,84.997215,1,75.682159,ending1,ending0,ending1,1,1
295,An arrow goes through the hole of a donut. An ...,74.971596,63.508228,62.587616,57.772198,2,63.508228,ending3,ending0,ending1,3,3
296,Another child moves around behind her. More ch...,50.764687,44.882004,57.364052,64.445938,3,44.882004,ending1,ending3,ending1,1,1
297,The woman exercise on front the ocean spinning...,70.271095,72.533035,68.238945,108.430801,1,72.533035,ending2,ending3,ending1,2,2


In [25]:
#calculate model pick vs human label:
comparison_2 = df['surprisal_smallest_number'] == df['label']
correct_predictions_2 = comparison_2.sum()
accuracy_2 = correct_predictions_2 / len(df)

In [26]:
accuracy_2

0.22073578595317725